In [1]:
# Imports
import os
from dotenv import load_dotenv
from pulsefire.clients import RiotAPIClient
from time import sleep
from supabase import create_client, Client
from tqdm import tqdm
from datetime import datetime, UTC


# Read .env and get RIOT API KEY
load_dotenv()
API_KEY = os.getenv("RIOT_API_KEY")

# Supabase env
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")

In [2]:
# Create client and test connection
db: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
try:
    # Simple query to test connection
    response = db.table('leaderboards').select("*").limit(1).execute()
    print("Successfully connected to Supabase!")
except Exception as e:
    print("Failed to connect to Supabase:")
    print(e)

Successfully connected to Supabase!


In [21]:
async def get_account_data_gmplus(region="euw1", queue="RANKED_SOLO_5x5"):
  """
    Sleep commands are there to avoid hitting the rate limit
    Current rate limit: 100 requests every 2 minutes
    Hence the sleep(1.5) -> ~80 requests every 2 minutes
  """
  async with RiotAPIClient(default_headers={"X-Riot-Token": API_KEY}) as client:
    try:
      challengers = await client.get_lol_league_v4_challenger_league_by_queue(region=region, queue=queue)
      grandmasters = await client.get_lol_league_v4_grandmaster_league_by_queue(region=region, queue=queue)
      print(f"Num of Challenger players: {len(challengers['entries'])}")
      print(f"Num of Grandmaster players: {len(grandmasters['entries'])}")
      print("---------------------------------")
    except Exception as e:
      print("Cannot get league data")
      print(e)
    # TODO fix Timestamp for the entries
    last_updated = datetime.now(UTC).isoformat()
    # Combine chall + gms
    leaderboard_entries = challengers['entries'] + grandmasters['entries']
    
    entry_id = 0 # running index
    accounts = [] # list of accounts
    print("Fetching account information...")
    # Progress bar
    pbar = tqdm(total=len(leaderboard_entries), desc="Getting account data")
    while entry_id < len(leaderboard_entries):
      try:
        entry = leaderboard_entries[entry_id]
        # Sleep 1.5 between api calls to stay below 100 calls / 2 min rate limit
        sleep(1.5)
        account = await client.get_account_v1_by_puuid(region="europe", puuid=entry['puuid'])
        
        # Create account dictionary
        account_data = {
          'player_id': entry['puuid'],
          'summoner_id': entry['summonerId'],
          'ign': account['gameName'],
          'tag': account['tagLine'],
          'region': region[:-1] if region[-1] == '1' else region, # For example euw has euw1 but kr just has kr, so conditionally remove last char
          'queue': 'solo' if queue == "RANKED_SOLO_5x5" else 'flex',
          'league': 'challenger' if entry_id + 1 <= 300 else 'grandmaster',
          'division': None,
          'rank': entry_id + 1,
          'lp': entry['leaguePoints'],
          'wins': entry['wins'],
          'losses': entry['losses'],
          'last_updated': last_updated
        }
        accounts.append(account_data)
        entry_id += 1
        pbar.update(1)
      except Exception as e:
        print("--------------ERROR--------------")
        print("Error in the loop:")
        print(e)
        print("--------------ERROR--------------")
        sleep(10)
  return accounts  

In [22]:
euw_gmplus = await get_account_data_gmplus(region="euw1", queue="RANKED_SOLO_5x5")

Num of Challenger players: 300
Num of Grandmaster players: 700
---------------------------------
Fetching account information...


Getting account data: 100%|██████████| 1000/1000 [28:21<00:00,  1.70s/it] 


In [23]:
response = (
    db.table('leaderboards')
    .upsert(euw_gmplus, on_conflict=["player_id"]) # Should match the db schema
    .execute()
)
print(response)

data=[{'player_id': 'vOjz0cXVbmT7n_Thwomn2WZrOcG3ZwJGxkm-RJSFzw3Z5UOhPcnwl3HUg3JKB_1tb9POA98YKBU7UQ', 'summoner_id': 'znbERNgVLgjoT4micxkbxKI5P374-3GlQAW1ixhYXHRa2I0H', 'ign': 'G2 SkewMond', 'tag': '3327', 'region': 'euw', 'queue': 'solo', 'league': 'challenger', 'division': None, 'rank': 1, 'lp': 2144, 'wins': 189, 'losses': 126, 'last_updated': '2025-02-28T08:51:26.228203+00:00'}, {'player_id': 'aEkcYOUDEVFH205vrntCxJoE5poXzkCWMcPa6QAcMHzCJY_zXVtxmi_rQ3y5JhuuV000QUGaqUGtjg', 'summoner_id': 'bbZbKVbAP_5Y-zoPXGpC2ZWnbirc76ByIwkLkFFOmjtQ3JNS61Ec5v9SUw', 'ign': 'J1HUIV', 'tag': '000', 'region': 'euw', 'queue': 'solo', 'league': 'challenger', 'division': None, 'rank': 2, 'lp': 1857, 'wins': 236, 'losses': 181, 'last_updated': '2025-02-28T08:51:26.228203+00:00'}, {'player_id': 'E4wWSXRhyhPDFJrB_kpivZEDGTXFQhtFmUg5_WRhMT_LPzZGu6g2eY8oNAExtxaOKYPZHITummlAgg', 'summoner_id': '6qog95sedFJ_4BL9i0LFaS5LOwkx-7dRdmMwJRm_GHSPaj8', 'ign': 'Odysseus', 'tag': '131', 'region': 'euw', 'queue': 'solo', '

In [24]:
response = (
  db.table('leaderboards')
  .select("*")
  .execute()
)

# response = (
#     db.table('leaderboards')
#     .select('*')
#     .eq('region', 'euw')  # filter where region = 'euw'
#     .order('rank', desc=False)  # order by rank ascending
#     .limit(10)  # limit to 10 results
#     .execute()
# )

In [25]:
print(len(response.data))

1000
